In [3]:
pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [36]:
## Help for yahou finance data

# t = yf.Ticker("AAPL")

# # list available expiries
# c = t.options
# print(expiries)

# # pick one expiry
# chain = t.option_chain(expiries[0])
# calls = chain.calls
# puts  = chain.puts

# # example: get a column with market prices
# calls[["contractSymbol","strike","lastPrice","bid","ask","impliedVolatility"]].head(5)

In [38]:
import yfinance as yf
import numpy as np
from datetime import datetime
from datetime import date

def get_spot(ticker):
    data = yf.download(ticker, period="5d", interval="1d", progress=False)
    if data.empty:
        raise ValueError("Ticker not available or no data.")
    else:
        price_column = "Adj Close" if "Adj Close" in data.columns else "Close"
    return float(data[price_column].iloc[-1])

# --- Block 1: choose underlying + store price ---
ticker = input("The underlying stock is: ").strip().upper()
S0 = get_spot(ticker)
print(f"The spot price for {ticker} = {S0:.2f} $")

# --- Block 2: show expiries ---
ticker_data = yf.Ticker(ticker)
expiries = ticker_data.options

print("\nAvailable expiries:")
for i, e in enumerate(expiries):
    print(f"[{i}] {e}")

The underlying stock is:  AAPL


The spot price for AAPL = 255.78 $

Available expiries:
[0] 2026-02-18
[1] 2026-02-20
[2] 2026-02-23
[3] 2026-02-25
[4] 2026-02-27
[5] 2026-03-06
[6] 2026-03-13
[7] 2026-03-20
[8] 2026-03-27
[9] 2026-04-17
[10] 2026-05-15
[11] 2026-06-18
[12] 2026-07-17
[13] 2026-08-21
[14] 2026-09-18
[15] 2026-11-20
[16] 2026-12-18
[17] 2027-01-15
[18] 2027-06-17
[19] 2027-12-17
[20] 2028-01-21
[21] 2028-03-17
[22] 2028-12-15


/tmp/ipykernel_162/1385031379.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data[price_column].iloc[-1])


In [ ]:
T_present=input("Is the desired maturity present in the list ? [y/n]").strip().lower()
if T_present in ('yes','y'):
    expiry=expiries[int(input("Select the index corresponding to the wanted maturity:").strip())]
    data_y = ticker_data.option_chain(expiry)
    expiry=datetime.strptime(expiry,"%Y-%m-%d").date()
    today = date.today()
    T=(expiry-today).days/365
    if T <= 0:
        raise ValueError("Expiry is in the past.")

    
    option_type=input("Select the option type:").lower().strip()
    df = data_y.calls if option_type in ("calls", "call",'c') else data_y.puts

    for i, K in enumerate(df["strike"]):
        print(f"[{i}] {round(K)}")

elif T_present in ('no','n'):
    print("Interpolated volatility from surface vol in comming ...")

else:
    print("Relaunch and select a correct input")


Is the desired maturity present in the list ? [y/n] y


In [42]:
K_present=input("Is the desired exercice price present in the list ? [y/n]").lower().strip()
if K_present in ("y","yes"):
    index_K=int(input("Select the index corresponding to the wanted maturity:").strip())
    ask_p=float(df.iloc[index_K]["ask"])
    bid_p=float(df.iloc[index_K]["bid"])
    K = float(df.iloc[index_K]["strike"])
    lastPrice=float(df.iloc[index_K]["lastPrice"])

    if ask_p > 0 and bid_p > 0:
        mkt_option_p=(ask_p+bid_p)/2
    else:
        mkt_option_p=lastPrice
    print(f"\nMarket option price is {mkt_option_p} $")
elif K_present in ('no','n'):
    print("\nInterpolated volatility from surface vol in comming ...")

else:
    print("\nRelaunch and select a correct input")

Is the desired exercice price present in the list ? [y/n] n



Interpolated volatility from surface vol in comming ...


In [43]:
r=float(input("The risk free rate (annual, decimal) is"))
q=float(input("The dividend yield (annual, decimal) is"))
print("\n!! Future work will give market-implied dividend yield and a risk free rate based on the maturity !!")

The risk free rate (annual, decimal) is 0.05
The dividend yield (annual, decimal) is 0.02



!! Future work will give market-implied dividend yield and a risk free rate based on the maturity !!


In [22]:
from scipy.optimize import brentq
from scipy.stats import norm

def d1_d2(S0,K,r,q,vol,T):
    d1=(np.log(S0/K)+(r-q+0.5*vol**2)*T)/(vol*np.sqrt(T))
    d2=d1-vol*np.sqrt(T)
    return d1, d2

def call_put(S0, K, T, r, q, vol,option_t):
    N=norm.cdf
    Dr=np.exp(-r*T)
    Dq=np.exp(-q*T)

    d1, d2 = d1_d2(S0, K, r, q, vol, T)
    
    call=S0*Dq*N(d1)-K*Dr*N(d2)
    put=K*Dr*N(-d2)-S0*Dq*N(-d1)
    if option_t in ("calls", "call",'c'):
        return call
    else:
        return put

def implied_vol_brent(S0, K, T, r, q, option_t, mkt_option_p):

    def errorf(vol):
        return call_put(S0, K, T, r, q, vol, option_t) - mkt_option_p

    return brentq(errorf, 1e-6, 5.0)

In [34]:
iv = implied_vol_brent(S0, K, T, r, q, option_type, mkt_option_p)
print(f"The implied vol for a {option_type} for {ticker} with an exercice price of {K:.0f} and a maturity date of {expiry} is {iv*100:.2f} %")

The implied vol for a c for AAPL with an exercice price of 300 and a maturity date of 2026-03-06 is 31.14 %
